In [0]:
"""File uploaded to /FileStore/tables/Damages_use.csv
File uploaded to /FileStore/tables/Endorse_use.csv
File uploaded to /FileStore/tables/Charges_use.csv
File uploaded to /FileStore/tables/Restrict_use.csv
File uploaded to /FileStore/tables/Primary_Person_use.csv
File uploaded to /FileStore/tables/Units_use.csv"""

Out[1]: 'File uploaded to /FileStore/tables/Damages_use.csv\nFile uploaded to /FileStore/tables/Endorse_use.csv\nFile uploaded to /FileStore/tables/Charges_use.csv\nFile uploaded to /FileStore/tables/Restrict_use.csv\nFile uploaded to /FileStore/tables/Primary_Person_use.csv\nFile uploaded to /FileStore/tables/Units_use.csv'

In [0]:
damages_df = spark.read.csv("/FileStore/tables/Damages_use.csv", header=True, inferSchema=True)
endorse_df = spark.read.csv("/FileStore/tables/Endorse_use.csv", header=True, inferSchema=True)
restrict_df = spark.read.csv("/FileStore/tables/Restrict_use.csv", header=True, inferSchema=True)
primary_person_df = spark.read.csv("/FileStore/tables/Primary_Person_use.csv", header=True, inferSchema=True)
units_df = spark.read.csv("/FileStore/tables/Units_use.csv", header=True, inferSchema=True)
charges_df = spark.read.csv("/FileStore/tables/Charges_use.csv", header=True, inferSchema=True)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [0]:
# This DF reports the number of males killed in accidents. This DF only takes the death of primary persons in consideration, as data for non-primary persons involved in the crash is not available.
primary_person_df.select(primary_person_df['CRASH_ID'],primary_person_df['UNIT_NBR'],primary_person_df['PRSN_NBR'],primary_person_df['PRSN_GNDR_ID'],primary_person_df['PRSN_INJRY_SEV_ID']).filter((primary_person_df['PRSN_GNDR_ID']=='MALE') & (primary_person_df["PRSN_INJRY_SEV_ID"]=="KILLED")).groupBy(primary_person_df["CRASH_ID"]).agg(count("*").alias("death_count")).sort(col("death_count").desc()).filter(col("death_count")>=2).show()

#This DF is an alternate solution, assuming missing data. Unable to filter on gender as units_table does not capture gender details
units_df.select('crash_id','unit_nbr','death_cnt').distinct().groupBy('crash_id').agg(sum('death_cnt').alias("tot_death_count")).filter(col("tot_death_count")>2).show()


+--------+-----------+
|CRASH_ID|death_count|
+--------+-----------+
|15379024|          2|
|15429998|          2|
+--------+-----------+

+--------+---------------+
|crash_id|tot_death_count|
+--------+---------------+
|15129823|              3|
|14952803|              4|
|15070158|              3|
+--------+---------------+



In [0]:
primary_person = primary_person_df.createOrReplaceTempView("primary_person")
units = units_df.createOrReplaceTempView("units")
damages= damages_df.createOrReplaceTempView("damages")
charges = charges_df.createOrReplaceTempView("charges")

In [0]:
spark.sql("""select crash_id, count(*) from primary_person where prsn_gndr_id='MALE' and PRSN_INJRY_SEV_ID
='KILLED' group by crash_id having count(*)>2""").show()

+--------+--------+
|crash_id|count(1)|
+--------+--------+
+--------+--------+



In [0]:
        crashes_with_male_deaths_df = primary_person_df.filter(
            (col("prsn_gndr_id") == "MALE") &
            (col("PRSN_INJRY_SEV_ID") == "KILLED")
        ).groupBy("crash_id").agg(count("*").alias("male_death_count"))

        # Filter for crashes with more than 2 male fatalities
        result_df = crashes_with_male_deaths_df.filter(col("male_death_count") > 2)

In [0]:
# This DF reports the number of motorcycles booked for crashes. Assumptions - Police bikes cannot be booked, Booked motorcycles have an entry in the charges dataset
units_df.join(charges_df,(units_df['CRASH_ID']==charges_df['CRASH_ID']) & (units_df['UNIT_NBR']==charges_df['UNIT_NBR']),"inner").filter((units_df["VEH_BODY_STYL_ID"]=='MOTORCYCLE') | (units_df["VEH_BODY_STYL_ID"]=='POLICE MOTORCYCLE')).select(units_df["CRASH_ID"]).distinct().count()

Out[9]: 396

In [0]:
units_df.filter

Out[10]: <bound method DataFrame.filter of DataFrame[CRASH_ID: int, UNIT_NBR: int, UNIT_DESC_ID: string, VEH_PARKED_FL: string, VEH_HNR_FL: string, VEH_LIC_STATE_ID: string, VIN: string, VEH_MOD_YEAR: string, VEH_COLOR_ID: string, VEH_MAKE_ID: string, VEH_MOD_ID: string, VEH_BODY_STYL_ID: string, EMER_RESPNDR_FL: string, OWNR_ZIP: string, FIN_RESP_PROOF_ID: string, FIN_RESP_TYPE_ID: string, VEH_DMAG_AREA_1_ID: string, VEH_DMAG_SCL_1_ID: string, FORCE_DIR_1_ID: string, VEH_DMAG_AREA_2_ID: string, VEH_DMAG_SCL_2_ID: string, FORCE_DIR_2_ID: string, VEH_INVENTORIED_FL: string, VEH_TRANSP_NAME: string, VEH_TRANSP_DEST: string, CONTRIB_FACTR_1_ID: string, CONTRIB_FACTR_2_ID: string, CONTRIB_FACTR_P1_ID: string, VEH_TRVL_DIR_ID: string, FIRST_HARM_EVT_INV_ID: string, INCAP_INJRY_CNT: int, NONINCAP_INJRY_CNT: int, POSS_INJRY_CNT: int, NON_INJRY_CNT: int, UNKN_INJRY_CNT: int, TOT_INJRY_CNT: int, DEATH_CNT: int]>

In [0]:
# This DF outputs the number of motorcycles booked for accidents. Future check - check if charge_id and unit_id are composite keys for this dataset

booked_units = charges_df.select("crash_id","unit_nbr").distinct()
units_df.join(
    booked_units,
    (units_df['CRASH_ID'] == booked_units['CRASH_ID']) & (units_df['UNIT_NBR'] == booked_units['UNIT_NBR']),
    "inner"
).filter(
    (units_df["VEH_BODY_STYL_ID"] == 'MOTORCYCLE') | (units_df["VEH_BODY_STYL_ID"] == 'POLICE MOTORCYCLE')
).select(units_df["CRASH_ID"],units_df['UNIT_NBR']).distinct().count()


# This DF is more straightforward
units_df.filter(col("VEH_BODY_STYL_ID").contains("MOTORCYCLE")).select("crash_id","unit_nbr").distinct().count()

Out[11]: 773

In [0]:
charges_df.select("crash_id","unit_nbr").count()

Out[12]: 116110

In [0]:
spark.sql("""with crash as (select crash_id, unit_nbr from primary_person where prsn_type_id="DRIVER" and PRSN_INJRY_SEV_ID='KILLED' and  PRSN_AIRBAG_ID = 'NOT DEPLOYED'), units as (select distinct crash_id, unit_nbr,VEH_MAKE_ID from units) select VEH_MAKE_ID, count(*) as Count from (select crash.crash_id, crash.unit_nbr,units.VEH_MAKE_ID from crash inner join units on crash.crash_id = units.crash_id) group by VEH_MAKE_ID order by Count limit 5""").show()

+-------------+-----+
|  VEH_MAKE_ID|Count|
+-------------+-----+
|        BUICK|    1|
|         HINO|    1|
|        ACURA|    1|
|INTERNATIONAL|    1|
|       TOYOTA|    1|
+-------------+-----+



In [0]:
# This SQL code outputs the top 5 vehicle makes of the cars present in the crashes in which the driver died and airbags did not deploy
spark.sql("""select veh_make_id, count(*) as cnt from (select distinct crash_id, unit_nbr,veh_make_id from units where units.crash_id in (select crash_id from primary_person where prsn_type_id="DRIVER" and PRSN_INJRY_SEV_ID='KILLED' and  PRSN_AIRBAG_ID = 'NOT DEPLOYED')) group by veh_make_id order by cnt desc limit 5""").show()

# This DF achieves the same result 

filtered_primary_person_df = primary_person_df.filter(
    (col("prsn_type_id") == "DRIVER") &
    (col("PRSN_INJRY_SEV_ID") == "KILLED") &
    (col("PRSN_AIRBAG_ID") == "NOT DEPLOYED")
).select("crash_id")

joined_df = units_df.join(filtered_primary_person_df,(units_df['crash_id'] == filtered_primary_person_df['crash_id']),'inner').select(units_df["crash_id"], "unit_nbr", "veh_make_id").distinct()

result_df = joined_df.groupBy("veh_make_id").agg(count("*").alias("cnt")).orderBy("cnt", ascending=False).limit(5)

result_df.show()

+-----------+---+
|veh_make_id|cnt|
+-----------+---+
|  CHEVROLET| 16|
|       FORD| 12|
|         NA|  7|
|      DODGE|  7|
|     NISSAN|  5|
+-----------+---+

+-----------+---+
|veh_make_id|cnt|
+-----------+---+
|  CHEVROLET| 16|
|       FORD| 12|
|         NA|  7|
|      DODGE|  7|
|     NISSAN|  5|
+-----------+---+



In [0]:
#This Spark SQL code outputs the number of vehicles involved in hit and runs where the driver had a valid drivers license
spark.sql("""select count(*) from primary_person inner join (select distinct crash_id,veh_hnr_fl from units) units_hnr on primary_person.crash_id = units_hnr.crash_id where prsn_type_id='DRIVER' and DRVR_LIC_CLS_ID != 'UNLICENSED'""" ).show()

#This DF achieves the same result 

units_distinct_df = spark.read.table("units").select("crash_id", "veh_hnr_fl").distinct()

# Join tables
joined_df = primary_person_df.join(
    units_distinct_df,
    primary_person_df['crash_id'] == units_distinct_df['crash_id'],
    'inner'
).filter(
    (col("prsn_type_id") == "DRIVER") &
    (col("DRVR_LIC_CLS_ID") != "UNLICENSED")
)

# Count the occurrences
result_df = joined_df.agg(count("*").alias("count"))

# Show the result
result_df.show()

+--------+
|count(1)|
+--------+
|  147956|
+--------+

+------+
| count|
+------+
|147956|
+------+



In [0]:
# This Spark SQL code outputs the state with the highest accidents in which women were not involved
#spark.sql("""select drvr_lic_state_id, count(*) as state_count from (select distinct crash_id, drvr_lic_state_id from primary_person where crash_id not in (select distinct crash_id from primary_person where prsn_gndr_id='FEMALE') and drvr_lic_cls_id !='OTHER/OUT OF STATE
#') group by drvr_lic_state_id order by state_count desc limit 1""").show()

#This DF achieves the same result
spark.sql("""select distinct crash_id, drvr_lic_state_id from primary_person where crash_id not in (select distinct crash_id from primary_person where prsn_gndr_id='FEMALE') and drvr_lic_cls_id !='OTHER/OUT OF STATE'""").count()

female_drivers = primary_person_df.filter(col("prsn_gndr_id")=='FEMALE').select('crash_id').distinct() 

primary_person_df.join(female_drivers,primary_person_df['crash_id']==female_drivers['crash_id'],"leftanti").select(primary_person_df["crash_id"],'DRVR_LIC_STATE_ID').distinct().groupBy(col('DRVR_LIC_STATE_ID')).agg(count('DRVR_LIC_STATE_ID').alias('state_count')).orderBy(col("state_count").desc()).limit(1).show()

# Main query to get drvr_lic_state_id counts
#result_df = non_female_crash_ids.filter(col('DRVR_LIC_CLS_ID')!='OTHER/OUT OF STATE').groupBy(col('DRVR_LIC_STATE_ID')).agg(count('DRVR_LIC_STATE_ID').alias('state_count')).orderBy(col("state_count").desc()).limit(1)
# Show the result
#result_df.show()

+-----------------+-----------+
|DRVR_LIC_STATE_ID|state_count|
+-----------------+-----------+
|            Texas|      34842|
+-----------------+-----------+



In [0]:
# This Spark SQL code outputs the 3-5th vehicle makes that cause the most accidents
#spark.sql("""select veh_make_id, sum(total_cas) as sum_cas from (select crash_id,veh_make_id,TOT_INJRY_CNT+DEATH_CNT as total_cas from (select distinct crash_id,TOT_INJRY_CNT,DEATH_CNT,veh_make_id from units where veh_make_id!='NA')) group by veh_make_id order by sum_cas desc limit 5 offset 2""").show()

#This DF outputs the same result

distinct_cas=units_df.filter(col("veh_make_id") != "NA").select("crash_id","veh_make_id","TOT_INJRY_CNT","DEATH_CNT").distinct()

total_cas_df = distinct_cas.select('crash_id','veh_make_id',(col('tot_injry_cnt')+col('death_cnt')).alias("tot_cas_cnt")).groupBy("veh_make_id").agg(sum('tot_cas_cnt').alias("sum_cas")).orderBy(col('sum_cas').desc()).limit(5)


total_cas_df.limit(5).subtract(total_cas_df.limit(2)).show()

+-----------+-------+
|veh_make_id|sum_cas|
+-----------+-------+
|     TOYOTA|   4149|
|      DODGE|   3098|
|     NISSAN|   3079|
+-----------+-------+



In [0]:
#top ethnicity for each vehicle body style
#spark.sql("""select prsn_ethnicity_id, veh_body_styl_id from (select PRSN_ETHNICITY_ID, veh_body_styl_id, row_number() over (partition by veh_body_styl_id order by cnt desc) as row_num from (select prsn_ethnicity_id, veh_body_styl_id, count(*) as cnt from (select p.crash_id, p.PRSN_ETHNICITY_ID
#, u.unit_nbr, u.veh_body_styl_id from primary_person p inner join 
#(select distinct crash_id, unit_nbr,VEH_BODY_STYL_ID from units where VEH_BODY_STYL_ID
#not in ('NA','OTHER  (EXPLAIN IN NARRATIVE)','NOT REPORTED')) u
##on p.crash_id = u.crash_id and p.unit_nbr = u.unit_nbr
#where p.prsn_ethnicity_id not in ("NA", "UNKNOWN")) group by PRSN_ETHNICITY_ID, veh_body_styl_id)) where row_num =1""").display()

# dataframe api code
filtered_units_df = units_df.filter(
    col("VEH_BODY_STYL_ID").isin("NA", "OTHER  (EXPLAIN IN NARRATIVE)", "NOT REPORTED") == False
).select("crash_id", "unit_nbr", "VEH_BODY_STYL_ID").distinct()

# Join tables
joined_df = primary_person_df.join(
    filtered_units_df,
    (primary_person_df['crash_id'] == filtered_units_df['crash_id']) & (primary_person_df['unit_nbr'] == filtered_units_df['unit_nbr'])
).filter(
    col("prsn_ethnicity_id").isin("NA", "UNKNOWN") == False
).select("PRSN_ETHNICITY_ID", "VEH_BODY_STYL_ID")

# Group by PRSN_ETHNICITY_ID and VEH_BODY_STYL_ID and count occurrences
grouped_df = joined_df.groupBy("PRSN_ETHNICITY_ID", "VEH_BODY_STYL_ID").agg(count("*").alias("cnt"))

# Use window function to assign row numbers based on the count
windowSpec = Window.partitionBy("VEH_BODY_STYL_ID").orderBy(col("cnt").desc())
ranked_df = grouped_df.withColumn("row_num", row_number().over(windowSpec))

# Filter rows with row number 1
result_df = ranked_df.filter(col("row_num") == 1).select("PRSN_ETHNICITY_ID", "VEH_BODY_STYL_ID")

# Display the result
result_df.show()

+-----------------+--------------------+
|PRSN_ETHNICITY_ID|    VEH_BODY_STYL_ID|
+-----------------+--------------------+
|            WHITE|           AMBULANCE|
|            BLACK|                 BUS|
|            WHITE|      FARM EQUIPMENT|
|            WHITE|          FIRE TRUCK|
|            WHITE|          MOTORCYCLE|
|            WHITE|NEV-NEIGHBORHOOD ...|
|            WHITE|PASSENGER CAR, 2-...|
|            WHITE|PASSENGER CAR, 4-...|
|            WHITE|              PICKUP|
|            WHITE|    POLICE CAR/TRUCK|
|            WHITE|   POLICE MOTORCYCLE|
|            WHITE|SPORT UTILITY VEH...|
|            WHITE|               TRUCK|
|            WHITE|       TRUCK TRACTOR|
|            WHITE|             UNKNOWN|
|            WHITE|                 VAN|
|            BLACK|   YELLOW SCHOOL BUS|
+-----------------+--------------------+



In [0]:
#analysis 8
#spark.sql("""select drvr_zip, count(*) as tot_count from (select crash_id,drvr_zip from (select crash_id, drvr_zip, row_number() over (partition by crash_id, drvr_zip order by unit_nbr) as row_num from primary_person) where row_num = 1 and crash_id in (select distinct crash_id from units where CONTRIB_FACTR_1_ID like '%ALCOHOL%' or CONTRIB_FACTR_2_ID like '%ALCOHOL%' or CONTRIB_FACTR_P1_ID like '%ALCOHOL%') and drvr_zip is not null ) group by drvr_zip order by tot_count desc limit 5""").show()
alcohol_crash_ids_df = units_df.filter(
    (col("CONTRIB_FACTR_1_ID").like("%ALCOHOL%")) |
    (col("CONTRIB_FACTR_2_ID").like("%ALCOHOL%")) |
    (col("CONTRIB_FACTR_P1_ID").like("%ALCOHOL%"))
).select("crash_id").distinct()
#dataframe code 
joined_df = primary_person_df.join(
    alcohol_crash_ids_df,
    primary_person_df['crash_id'] == alcohol_crash_ids_df['crash_id']
).select(primary_person_df["crash_id"], "drvr_zip", "unit_nbr")

# Use window function to assign row numbers based on crash_id and drvr_zip
windowSpec = Window.partitionBy("crash_id", "drvr_zip").orderBy("unit_nbr")
ranked_df = joined_df.withColumn("row_num", row_number().over(windowSpec))

# Filter rows with row number 1 and non-null drvr_zip
filtered_df = ranked_df.filter((col("row_num") == 1) & (col("drvr_zip").isNotNull()))

# Group by drvr_zip and count occurrences
result_df = filtered_df.groupBy("drvr_zip").agg(count("*").alias("tot_count")).orderBy(col("tot_count").desc()).limit(5)

# Show the result
result_df.show()

+--------+---------+
|drvr_zip|tot_count|
+--------+---------+
|   76010|       65|
|   78521|       58|
|   75067|       58|
|   78542|       46|
|   77084|       46|
+--------+---------+



In [0]:
spark.sql("""select crash_id,  (select distinct crash_id from units where CONTRIB_FACTR_1_ID like '%ALCOHOL%' or CONTRIB_FACTR_2_ID like '%ALCOHOL%' or CONTRIB_FACTR_P1_ID like '%ALCOHOL%')""").show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-72306166561033>:1
----> 1 spark.sql("""select crash_id,  (select distinct crash_id from units where CONTRIB_FACTR_1_ID like '%ALCOHOL%' or CONTRIB_FACTR_2_ID like '%ALCOHOL%' or CONTRIB_FACTR_P1_ID like '%ALCOHOL%')""").show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_java_column(lit(v)) for k, v in (args or {}).items()}
-> 1387     return DataFr

In [0]:
#analysis 9 
spark.sql("""select distinct crash_id from units where crash_id not in (select crash_id from damages) and (VEH_DMAG_SCL_1_ID in ('DAMAGED 5','DAMAGED 6','DAMAGED 7 HIGHEST') or VEH_DMAG_SCL_2_ID
in ('DAMAGED 5','DAMAGED 6','DAMAGED 7 HIGHEST')) and FIN_RESP_TYPE_ID like '%INSURANCE%'""").count()


#dataframe code
# Filter out distinct crash_id where crash_id is not in damages and VEH_DMAG_SCL_1_ID or VEH_DMAG_SCL_2_ID contains specified values
filtered_units_df = units_df.filter(
    (
        (col("VEH_DMAG_SCL_1_ID").isin("DAMAGED 5", "DAMAGED 6", "DAMAGED 7 HIGHEST")) |
        (col("VEH_DMAG_SCL_2_ID").isin("DAMAGED 5", "DAMAGED 6", "DAMAGED 7 HIGHEST"))
    ) &
    (col("FIN_RESP_TYPE_ID").like("%INSURANCE%"))
)

no_damage_df = filtered_units_df.join(damages_df, filtered_units_df['crash_id']==damages_df['crash_id'],'leftanti')

# Count the distinct crash_ids
count = no_damage_df.select("crash_id").distinct().count()

print(count)

8849


In [0]:
#analysis 10 
spark.sql("""with top_state as (select veh_lic_state_id, count(*) as state_count from (select distinct crash_id,veh_lic_state_id from units) where veh_lic_state_id not in ('Unknown','NA') group by veh_lic_state_id order by state_count desc limit 25), top_colour as (select veh_color_id, count(*) as color_count from (select distinct crash_id,unit_nbr, veh_color_id from units where veh_color_id != 'NA') group by veh_color_id order by color_count desc limit 10)
select units.VEH_MAKE_ID, count(*) as make_count from primary_person inner join (select distinct crash_id,VEH_MAKE_ID,unit_nbr, veh_color_id, veh_lic_state_id from units) units on primary_person.crash_id = units.crash_id 
inner join 
(select * from charges where charge like '%SPEED%') charge
on primary_person.crash_id = charge.crash_id
where primary_person.DRVR_LIC_CLS_ID like '%CLASS%' and units.veh_color_id in (select veh_color_id from top_colour) and units.veh_lic_state_id in (select veh_lic_state_id from top_state)
group by units.VEH_MAKE_ID
order by make_count desc
limit 5
""").show()



+-----------+----------+
|VEH_MAKE_ID|make_count|
+-----------+----------+
|       FORD|     18021|
|  CHEVROLET|     15917|
|     TOYOTA|     11286|
|      DODGE|      7423|
|     NISSAN|      6945|
+-----------+----------+



In [0]:
#df code for analysis 10 
top_state_df = units_df.filter(~col("veh_lic_state_id").isin("NA")) \
    .select("crash_id", "veh_lic_state_id").distinct() \
    .groupBy("veh_lic_state_id").agg(count("*").alias("state_count")) \
    .orderBy(col("state_count").desc()).limit(25)

top_colour_df = units_df.filter(col("veh_color_id") != "NA") \
    .select("crash_id", "unit_nbr", "veh_color_id").distinct() \
    .groupBy("veh_color_id").agg(count("*").alias("color_count")) \
    .orderBy(col("color_count").desc()).limit(10)

# Main Query
make_count_df = primary_person_df.join(
    units_df.select("crash_id", "VEH_MAKE_ID", "unit_nbr", "veh_color_id", "veh_lic_state_id").distinct(),
    "crash_id"
).join(
    charges_df.filter(col("charge").like("%SPEED%")).select("crash_id"),
    "crash_id"
).join(
    top_colour_df.select("veh_color_id"),
    "veh_color_id"
).join(
    top_state_df.select("veh_lic_state_id"),
    "veh_lic_state_id"
).filter(
    col("DRVR_LIC_CLS_ID").like("%CLASS%")
).groupBy("VEH_MAKE_ID").agg(count("*").alias("make_count")) \
.orderBy(col("make_count").desc()).limit(5)

make_count_df.show()

+-----------+----------+
|VEH_MAKE_ID|make_count|
+-----------+----------+
|       FORD|     18021|
|  CHEVROLET|     15917|
|     TOYOTA|     11286|
|      DODGE|      7423|
|     NISSAN|      6945|
+-----------+----------+



In [0]:
spark.sql("""select)

  File <command-2118443724326295>:1
    spark.sql("""select)
                        ^
SyntaxError: EOF while scanning triple-quoted string literal
